## Definición del problema

El 15 de abril de 1912 el Titanic RMS se hundió después de haber chocado con un iceberg. Desafortunadamente, no había suficientes botes salvavidas para todos los que estaban a bordo, lo cual resultó en el fallecimiento de 1502 de los 2224 pasajeros y tripulantes. Si bien hubo algún elemento de suerte involucrado en la supervivencia, parece que algunos grupos de personas tenían más probabilidades de sobrevivir que otros.

Nuestro trabajo consiste en construir un modelo de predicción que responda a la siguiente pregunta: ¿qué tipo de personas tenían más probabilidades de sobrevivir? Utilizando los datos de los pasajeros que están en los archivos train.csv y test.csv, disponibles en la plataforma Kaggle: Titanic - Machine Learning from Disaster

El archivo train.csv contiene detalles de un subconjunto de 891 pasajeros, en el cual se revela cuáles de estos pasajeros sobrevivieron o no al accidente (ground truth). Por otro lado, el archivo test.csv contiene información similar pero no revela el resultado de supervivencia de cada pasajero. Este último archivo se utilizará para probar la efectividad de nuestro modelo de predicción. Utilizando los patrones encontrados en los datos de train.csv, nosotros necesitamos predecir si los otros 418 pasajeros a bordo (test.csv) sobrevivieron.

Por lo tanto, en resumidas cuentas, nuestro objetivo es crear un modelo que predice cuáles pasajeros sobrevivieron al naufragio del Titanic. La métrica de éxito para este reto corresponde al porcentaje de similitud que tienen los resultados de nuestro modelo comparado con los datos reales.


In [709]:
from google.colab import drive
drive.mount('/content/drive') #No es necesario si el archivo esta guardado localmente

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [710]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd
import math

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [711]:
train_df = pd.read_csv('/content/drive/MyDrive/TC3006C.101_OpenAIGPT/Reto/train.csv') #Cambiar ruteo con de acuerdo a la ubicacion el archivo train.csv en su dispositivo
test_df = pd.read_csv('/content/drive/MyDrive/TC3006C.101_OpenAIGPT/Reto/test.csv')  #Cambiar ruteo con de acuerdo a la ubicacion el archivo test.csv en su dispositivo
train_df.head()

# Combine these datasets to run certain operations on both datasets together
combine = [train_df, test_df]

### Análisis por descripción de datos

* ¿Cuáles atributos están disponibles en los datasets?
* ¿Cuáles atributos son categóricos?
* ¿Cuáles atributos son numéricos?
* ¿Cuáles atributos son una mezcla de tipos de datos?
* ¿Cuáles atributos pueden contenter errores?
* ¿Cuáles atributos contienen espacios en blanco, valores nulos o valores faltantes?
* ¿Cuáles son los tipos de datos para cada atributo?
* ¿Cuál es la distribución de atributos numéricos de los valores de las muestras?

Primero revisamos qué características están en los conjuntos de datos y qué tipo de información contienen. Esto nos ayuda a entender qué atributos son categóricos, cuáles son números y cuáles son una mezcla de ambos. A partir de eso, podremos analizar cuáles son los atributos más que deducimos que son los más importantes para descifrar la probabilidad de supervivencia de los pasajeros al igual que poder arreglar algún error que tenga esos datos. Esos que nos consideramos lo suficientemente importantes serán descartados.

In [712]:
# ¿Cuales atributos están disponibles en los datasets?
attributes = train_df.columns.values
print(attributes)
print("\n")
# Identificar los tipos de datos para cada atributo
datatypes = pd.DataFrame(train_df.dtypes)
print(datatypes)

['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch'
 'Ticket' 'Fare' 'Cabin' 'Embarked']


                   0
PassengerId    int64
Survived       int64
Pclass         int64
Name          object
Sex           object
Age          float64
SibSp          int64
Parch          int64
Ticket        object
Fare         float64
Cabin         object
Embarked      object


## Limpieza del Conjunto de Datos

En este apartado se descargan los datasets pertinentes al reto y se realiza un análisis de los datos antes de limpiarlo. Esta etapa comprende de los siguientes elementos:

1. Limpia de datos con el uso de herramientas ETL
2. Explicación y documentación de cada decisión tomada sobre cómo limpiar los atributos y valores. Se explica también cuáles son las variables que se creen que son más relevantes para el modelo y por qué.
3. Aplicación de las transformaciones necesarias a los datos usando herramientas de ETL.
4. Explicación y documentación de cada decisión que hayamos tomado sobre cómo transformar cada variable.



### Identificación de duplicados
Considerando que la eliminación y gestión adecuada de duplicados puede tener un impacto significativo en la calidad y la integridad de los datos se realizó una revisión sobre los siguientes elementos: registros, identificadores de pasajeros, nombres y tickets.

Resultado: El único elemento en el que se identificaron duplicados es "tickets". En el dataframe de entrenamiento se encontró que el 23.6% de los tickets están duplicados, mientras que en el dataframe de prueba se encontró un 13.2%. Por lo tanto, esto indica que el atributo *Tickets* puede que no sea muy bueno para nuestro modelo.

In [713]:
# Funcion que revisa si hay elementos duplicados en el dataframe
def checar_duplicados(dataframe, dataframe_name):
    duplicate_rows = dataframe[dataframe.duplicated()]
    duplicate_ids = dataframe[dataframe['PassengerId'].duplicated()]
    duplicate_names = dataframe[dataframe['Name'].duplicated()]
    duplicate_tickets = dataframe[dataframe['Ticket'].duplicated()]

    print(dataframe_name + " dataframe")

    if duplicate_rows.shape[0] > 0:
        dp = (len(duplicate_rows)*100)/len(dataframe)
        print(f"Registros duplicados encontrados: {dp}%")
    else:
        print("No se encontraron registros duplicados.")

    if duplicate_ids.shape[0] > 0:
        di = (len(duplicate_ids)*100)/len(dataframe)
        print(f"Identificadores duplicados encontrados: {di}%")
    else:
        print("No se encontraron identificadores duplicados.")

    if duplicate_names.shape[0] > 0:
        dn = (len(duplicate_names)*100)/len(dataframe)
        print(f"Nombres duplicados encontrados: {dn}%")
    else:
        print("No se encontraron nombres duplicados.")

    if duplicate_tickets.shape[0] > 0:
        dt = (len(duplicate_tickets)*100)/len(dataframe)
        print(f"Tickets duplicados encontrados: {dt}%")
    else:
        print("No se encontraron tickets duplicados.")

checar_duplicados(train_df, "Train")
print("\n")
checar_duplicados(test_df, "Test")

Train dataframe
No se encontraron registros duplicados.
No se encontraron identificadores duplicados.
No se encontraron nombres duplicados.
Tickets duplicados encontrados: 23.569023569023567%


Test dataframe
No se encontraron registros duplicados.
No se encontraron identificadores duplicados.
No se encontraron nombres duplicados.
Tickets duplicados encontrados: 13.157894736842104%


### Identificación y tratamiento de valores nulos

Para asegurar una limpieza y mantenimiento correcto de la información es importante poder revisar cúales atributos pueden tener datos nulos con fin de ver si es posible modificar los datos o descartarlos.


Empezamos por identificando que atributos dentro del dataframe contienen datos nulos y los sumamos con fin de visualizarlo.

In [714]:
# Funcion que identifica la cantidad de valores nulos por atributo de dataframe
def identificar_nulos(dataframe, dataframe_name):
  print(dataframe_name + " dataframe")
  print(dataframe.isnull().sum())

identificar_nulos(train_df, "Train")
print("\n")
identificar_nulos(test_df, "Test")


Train dataframe
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


Test dataframe
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


Dada la importancia fundamental que la edad tiene en la evaluación de las posibilidades de supervivencia de los pasajeros, es imperativo mantener y, en caso de ausencia de esta información, ajustar de manera adecuada dicho atributo. Esto implica tener en consideración la edad promedio asociada a cada título de los pasajeros. Tomemos como ejemplo el título "Master" otorgado a los niños menores de edad; en situaciones donde la edad de un pasajero con este título no esté disponible, es razonable inferir una edad sustituta que sea coherente con el contexto y permita llenar este vacío de datos de manera coherente.

In [715]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Como mencionado anteriormente, todos esos valores de edad que sean nulos seran remplazados por el valor promedio del titulo del pasajero. Por ejemplo: un "Master" tendra edad una de 4 y una "Mrs" tendra una edad de 35.

### Tranformar tipo de variables para leer datos int

Con el propósito de homogeneizar los datos en el dataframe y convertirlos en valores numéricos, se procede a sustituir las designaciones "male" o "female" por 1 y 0, respectivamente. Además, las localizaciones de embarque representadas por "C", "S" y "Q" son transformadas en los valores numéricos 1, 2 y 3, respectivamente. De esta manera, se logra una estructura coherente y compatible para el modelo.

In [716]:
def get_avg_age_title(dataframe):
  # Identificamos los atributos que tienen valores nulos
  null_counts = dataframe.isnull().sum()

  # Calculamos el promedio de edad por título al igual que agregamos el atributo
  # "Title" (Mr, Ms, Mrs, Master, etc.) al dataframe dado que es la unica pieza de información relevante.
  dataframe["Title"] = dataframe["Name"].str.extract(' ([A-Za-z]+)\.', expand=False)
  average_age_by_title = dataframe.groupby("Title")["Age"].mean().astype(int)

  # Regresamos un diccionario mostrando el promedio de edad por titulo
  # A partir de esto, podremos rellenar esos datos de edad nulos.
  return average_age_by_title.to_dict()

promedio_edad_titulo_train = get_avg_age_title(train_df)

train_df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr


In [717]:
def estadarizar_atributos(dataframe, dataframe_name, promedio_edad):
    # Mapeo de valores en la columna 'Sex' a valores enteros
    mapeo_genero = {'male': 1, 'female': 0}
    dataframe['Sex'] = dataframe['Sex'].map(mapeo_genero)

    # Mapeo de valores en la columna 'Embarked' a valores enteros
    mapeo_embarcada = {'C': 1, 'S': 2, 'Q': 3}
    dataframe['Embarked'] = dataframe['Embarked'].map(mapeo_embarcada)
    # Replazar valores nulos con 2 en embarked
    dataframe['Embarked'].fillna(1, inplace = True)
    #Convertir de float a int
    dataframe['Embarked'] = dataframe['Embarked'].astype(int)

    # Mapeao de valores en la columna 'Fare' a valores enteros
    dataframe['Fare'].fillna(dataframe['Fare'].dropna().median(), inplace=True)

    # Create FareBand
    dataframe['FareBand'] = pd.qcut(dataframe['Fare'], 4)

    # Convert the Fare feature to ordinal values based on the FareBand.
    dataframe.loc[ dataframe['Fare'] <= 7.91, 'Fare'] = 0
    dataframe.loc[(dataframe['Fare'] > 7.91) & (dataframe['Fare'] <= 14.454), 'Fare'] = 1
    dataframe.loc[(dataframe['Fare'] > 14.454) & (dataframe['Fare'] <= 31), 'Fare'] = 2
    dataframe.loc[ dataframe['Fare'] > 31, 'Fare'] = 3
    dataframe['Fare'] = dataframe['Fare'].astype(int)

    # Drop the FareBand column
    dataframe = dataframe.drop(['FareBand'], axis=1)

    dataframe["Title"] = dataframe["Name"].str.extract(' ([A-Za-z]+)\.', expand=False)
    dataframe['Age'] = dataframe.apply(lambda row: promedio_edad.get(row['Title'], np.nan) if np.isnan(row['Age']) else row['Age'], axis=1)
    #Redondear age en caso de que sea puro decimal
    dataframe['Age'] = dataframe['Age'].apply(math.floor)

    '''Mapear 'Embarked' valores (C: 1, S: 2, Q: 3)
    dataframe['Embarked'] = dataframe['Embarked'].map({"C": 1, "S": 2, "Q": 3})
    # Replazar valores nulos con 2 en embarked
    #dataframe['Embarked'].fillna(1, inplace = True)
    dataframe['Embarked'] = dataframe['Embarked'].apply(lambda x: 1 if pd.isnull(x['Embarked']) else x['Embarked'])
    #Convertir de float a int
    dataframe['Embarked'] = dataframe['Embarked'].astype(int)'''

    return dataframe

train_df = estadarizar_atributos(train_df, 'Train', promedio_edad_titulo_train)
test_df = estadarizar_atributos(test_df, 'Test', promedio_edad_titulo_train)


In [718]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",1,22,1,0,A/5 21171,0,NaN,2,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38,1,0,PC 17599,3,C85,1,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",0,26,0,0,STON/O2. 3101282,1,NaN,2,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35,1,0,113803,3,C123,2,Mrs
4,5,0,3,"Allen, Mr. William Henry",1,35,0,0,373450,1,NaN,2,Mr


### Eliminación de atributos

Los atributos que consideramos eliminar de nuestros conjuntos de datos son los siguientes:
* Name: No lo necesitamos para el modelo debido a que no tiene una directa correlación con la supervivencia de los pasajeros. Se podría utilizar como identificador de los registros, pero para esto ya tenemos el atributo "PassengerId".
* Title: Solo lo necesitamos para estimar la edad para los pasajeros que no tenían registrado este atributo y reemplazar los valores nulos de acuerdo a este estimado.
* Cabin: No conviene utilizarlo debido a su gran porcentaje de valores nulos. En el dataset de entrenamiento el 77.1% de sus valores son nulos, mientras que en el dataset de prueba correponde a 78.2%. Además, los valores de esta columna son cadenas de texto que difícilmente se pueden traducir a un valor numérico.
* Ticket: No conviene utilizarlo debido a que es un valor con tipos de datos mezclados y no se puede simplificar a un valor numérico. Además cuenta con un considerabel porcentaje de valores nulos y no creemos que tenga una correlación directa con la supervivencia de los pasajeros.

In [719]:
def eliminar_atributos(dataframe):
  dataframe = dataframe.drop(['Name', 'Cabin', 'Ticket', 'Title'], axis='columns')
  return dataframe

train_df = eliminar_atributos(train_df)
test_df = eliminar_atributos(test_df)

train_df.head()


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22,1,0,0,2
1,2,1,1,0,38,1,0,3,1
2,3,1,3,0,26,0,0,1,2
3,4,1,1,0,35,1,0,3,2
4,5,0,3,1,35,0,0,1,2


###Generar archivo csv para entrenar el modelo

In [720]:
# Creación de atributo Fam
train_df['Fam'] = train_df['SibSp'] + train_df['Parch']

# Seleccionar columnas necesarias
selected_columns = ['PassengerId', 'Pclass', 'Age', 'Sex', 'Fam', 'Fare', 'Embarked']
df = train_df[selected_columns]

selected_columns_results = ['PassengerId', 'Survived']
df_results = train_df[selected_columns_results]

# Guardar a archivo csv
df.to_csv('train_data.csv', index=False)
df_results.to_csv('train_results.csv', index=False)

print("Archivo CSV 'train_data.csv' guardado exitosamente.")

Archivo CSV 'train_data.csv' guardado exitosamente.


### Ejemplo de modelo final


In [721]:
y = train_df["Survived"]

#Ejemplo del tutorial para buscar atributos importantes
#Establecemos el arbol y que atributos consideramos que son necesarios

features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_df[features])
X_test = pd.get_dummies(test_df[features])

#Creamos el modelo y utilizamos la funcion de RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_df.PassengerId, 'Survived': predictions})
output.to_csv('resultados.csv', index=False)